<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Fine-tuning Mistral on your own data 🤙

Welcome!

In this notebook and tutorial, we will fine-tune the [Mistral 7B](https://github.com/mistralai/mistral-src) model - which outperforms Llama 2 13B on all tested benchmarks - ***on your own data!***

In [34]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import platform
print(platform.platform())
import torch
print(torch.__version__)
!nvcc --version

Tue Jan  9 15:26:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   38C    P8              13W / 500W |    317MiB / 24564MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Before we begin: A note on OOM errors

If you get an error like this: `OutOfMemoryError: CUDA out of memory`, tweak your parameters to make the model less computationally intensive. I will help guide you through that in this guide, and if you have any additional questions you can reach out on the [Discord channel](https://discord.gg/RN2a436M73) or on [X](https://x.com/harperscarroll).

To re-try after you tweak your parameters, open a Terminal ('Launcher' or '+' in the nav bar above -> Other -> Terminal) and run the command `nvidia-smi`. Then find the process ID `PID` under `Processes` and run the command `kill [PID]`. You will need to re-start your notebook from the beginning. (There may be a better way to do this... if so please do let me know!)

## Let's begin!
### 0. Preparing data

Before you check out a GPU, prepare your dataset for loading and training.

To prepare your dataset for loading, all you need are two `.jsonl` files structured something like this:
```
{"input": "What color is the sky?", "output": "The sky is blue."}
{"input": "Where is the best place to get cloud GPUs?", "output": "Brev.dev"}
```
If you choose to model your data as input/output pairs, you'll want to use something like the second `formatting_func` below, which will will combine all your features into one input string.

As you can see below, I have `notes.jsonl` for my `train_dataset` and `notes_validation.jsonl` for my `eval_dataset`.

I used Exporter, a free local-only app, to export my Apple Notes to `.txt` files, and then I wrote a script to process each note into one `.jsonl` file. Note that for this script, ChatGPT can help out a LOT if you tell it how your data is currently formatted, how you'd like it to be formatted, and ask it to write a script in a certain language you know well (for any debugging) to do so. I also broke up my journal entries so the training sample vector length was smaller (see the discussion on `max_length` and the data visualization below). I broke it into pieces so that contexts were encapsulated entirely, since I did want the model to understand context about my life. My data were ultimately formatted as:

```json
{"note": "journal-entry-for-model-to-predict"}
{"note": "journal-entry-for-model-to-predict-1"}
{"note": "journal-entry-for-model-to-predict-2"}
```

### 1. Load Dataset on a GPU

I used a GPU and dev environment from [brev.dev](https://brev.dev). The whole thing cost me $1 using a 1xA10G 24GB. Click the badge below to get your preconfigured instance:

[![](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdeploynavy.svg)](https://console.brev.dev/environment/new?instance=A10G:g5.xlarge&diskStorage=256&name=mistral-finetune-own-data&file=https://github.com/brevdev/notebooks/raw/main/mistral-finetune-own-data.ipynb&python=3.10&cuda=12.1.1)

A single A10G (as linked) with 24GB GPU Memory was enough for me. You may need more GPUs and/or Memory if your sequence max_length is larger than 512.

Once you've checked out your machine and landed in your instance page, select the specs you'd like (I used **Python 3.10 and CUDA 12.1.1**; these should be preconfigured for you if you use the badge above) and click the "Build" button to build your verb container. Give this a few minutes.

A few minutes after your model has started Running, click the 'Notebook' button on the top right of your screen once it illuminates (you may need to refresh the screen). You will be taken to a Jupyter Lab environment, where you can upload this Notebook.


Note: You can connect your cloud credits (AWS or GCP) by clicking "Org: " on the top right, and in the panel that slides over, click "Connect AWS" or "Connect GCP" under "Connect your cloud" and follow the instructions linked to attach your credentials.

In [1]:
model_checkpoint = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\model\\deepseek-aideepseek-coder-6.7b-instruct"
dataset_directory = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\Fine_Tune\\Datasets\\Saved_datasets\\shawhin-imdb-truncated"
dataset_name = 'shawhin/imdb-truncated'            
metrics_dir = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\Fine_Tune\\evaluate\\metrics\\accuracy\\accuracy.py"
#model_save_path = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\Fine_Tune\\Trained_models\\distilbert-base-uncased\\"

In [2]:
# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

#*********************************************************************************************************
#**                                                Load Data                                            **
#*********************************************************************************************************
# load dataset
dataset_directory = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\Fine_Tune_LLMs\\mcal-guru\\Fine_Tune\\Datasets\\Custom_datasets\\DUT\\"
train_path = dataset_directory + 'final_train.jsonl'
valid_path = dataset_directory + 'final_validation.jsonl'

In [3]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files=train_path, split='train')
eval_dataset = load_dataset('json', data_files=valid_path, split='train')

C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLMs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Formatting prompts
Then create a formatting function to structure training examples as prompts.

In [3]:
def formatting_func(example):
    text = f"### The following is a note by Eevee the Dog: {example['note']}"
    return text 

In [4]:
def formatting_func(example):
    text = f"### Read the below Input Function, then generate the appropriate Output Test Case.\n ### Input Function: {example['Input']}\n ### Output Test Case: {example['Output']}"
    return text

### 2. Load Base Model

Let's now load Mistral - mistralai/Mistral-7B-v0.1 - using 4-bit quantization!

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
print(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    device_map='auto',
    use_cache=False,
    quantization_config=nf4_config
    )#, quantization_config=bnb_config)
#model = AutoModelForCausalLM.from_pretrained(model_checkpoint, quantization_config=bnb_config)

C:\Users\rvc_sw_mss1_common\Desktop\test\mcal-guru\model\deepseek-aideepseek-coder-6.7b-instruct
bin C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLMs\venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.27s/it]


In [8]:
import gc
import torch
def clear_torch_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [9]:
clear_torch_cache()

In [10]:
def generate_response(prompt, model, tokenizer):
    with torch.no_grad():
        encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
        model_inputs = encoded_input.to("cuda")
    
        generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, repetition_penalty=1.15, pad_token_id=tokenizer.eos_token_id)
        decoded_output = tokenizer.batch_decode(generated_ids)
    clear_torch_cache()

    return decoded_output[0].replace(prompt, "")

In [13]:
#num_devices = torch.cuda.device_count()
#
#device_ids = [i for i in range(num_devices)]
#print(device_ids)
#num_devices = torch.cuda.device_count()
#
#device_ids = [i for i in range(num_devices)]
#model = torch.nn.DataParallel(model, device_ids=device_ids)
#model = model.cuda()

### 3. Tokenization

Set up the tokenizer. Add padding on the left as it [makes training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa).


For `model_max_length`, it's helpful to get a distribution of your data lengths. Let's first tokenize without the truncation/padding, so we can get a length distribution.

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Reformat the prompt and tokenize each sample:

In [13]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map: 100%|████████████████████████████████████████████████████████████████| 597/597 [00:03<00:00, 189.51 examples/s]


Let's get a distribution of our dataset lengths, so we can determine the appropriate `max_length` for our input tensors.

From here, you can choose where you'd like to set the `max_length` to be. You can truncate and pad training examples to fit them to your chosen size. Be aware that choosing a larger `max_length` has its compute tradeoffs.

I'm using my personal notes to train the model, and they vary greatly in length. I spent some time cleaning the dataset so the samples were about the same length, cutting up individual notes if needed, but being sure to not cut in the middle of a word or sentence.

Now let's tokenize again with padding and truncation, and set up the tokenize function to make labels and input_ids the same. This is basically what [self-supervised fine-tuning is](https://neptune.ai/blog/self-supervised-learning).

In [12]:
max_length = 2500 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt2)

Map: 100%|████████████████████████████████████████████████████████████████| 597/597 [00:03<00:00, 154.55 examples/s]


Check that `input_ids` is padded on the left with the `eos_token` (2) and there is an `eos_token` 2 added to the end, and the prompt starts with a `bos_token` (1).

In [14]:
print(tokenized_train_dataset[1]['input_ids'])

[32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021, 32021

Now all the samples should be the same length, `max_length`.

In [30]:
#plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

### How does the base model do?

Optionally, you can check how Mistral does on one of your data samples. For example, if you have a dataset of users' biometric data to their health scores, you could test the following `eval_prompt`:

In [18]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    add_bos_token=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
eval_prompt = """
Read the below Input Function, then generate the appropriate Output Test Case.\n ### Input Function: 
** Function Name        : Adc_AdcDeInit
** Service ID           : NA
** Description          : This internal function performs the de-initialization of the device-specific ADC hardware
**                        registers.
** Sync/Async           : Synchronous
** Re-entrancy          : Non-Reentrant
** Input Parameters     : LucHwUnitIndex
** InOut Parameters     : None
** Output Parameters    : None
** Return Parameter     : None
** Preconditions        : None
** Global Variables     : Adc_GpHwUnitConfig
** Functions invoked    : Adc_HwTriggerDeInit, Adc_TrackHoldDeInit, Adc_DeInitWaitTime
** Registers Used       : ADCXnDFASENTSGER, ADCXnSDVCLMINTER, ADCXnSGDIAGPCCR0, ADCXnSGDIAGPCCR1, ADCXnSGDIAGVCR0,
                        ADCXnSGDIAGVCR1, ADCXnSGDIAGVCR2, ADCXnSTPDCR0, ADCXnSTPDCR1, ADCXnSTPDCR2, ADCXnSGDIAGCR
** Reference ID         : ADC_DUD_ACT_085, ADC_DUD_ACT_085_REG001, ADC_DUD_ACT_085_REG002,
** Reference ID         : ADC_DUD_ACT_085_REG003, ADC_DUD_ACT_085_REG004, ADC_DUD_ACT_085_REG005,
** Reference ID         : ADC_DUD_ACT_085_REG006, ADC_DUD_ACT_085_REG007, ADC_DUD_ACT_085_REG008,
** Reference ID         : ADC_DUD_ACT_085_REG009, ADC_DUD_ACT_085_REG010, ADC_DUD_ACT_085_REG011
#if (ADC_DEINIT_API == STD_ON)

#define ADC_START_SEC_PRIVATE_CODE
#include "Adc_Mapping.h"                                                                                                /* PRQA S 5087 # JV-01 */

FUNC(void, ADC_PRIVATE_CODE) Adc_AdcDeInit(const uint8 LucHwUnitIndex)                                                  /* PRQA S 1532 # JV-01 */
{
  P2CONST(Adc_HwUnitConfigType, AUTOMATIC, ADC_CONFIG_DATA) LpHwUnitConfig;
  P2VAR(volatile Adc_ConfigRegisters, AUTOMATIC, REGSPACE) LpAdcRegisters;

  /* Initialize the local variable for HW unit configuration */
  LpHwUnitConfig = &Adc_GpHwUnitConfig[LucHwUnitIndex];                                                                 /* PRQA S 2824 # JV-01 */
  /* Read the user base configuration address of the HW unit */
  LpAdcRegisters = LpHwUnitConfig->pHwUnitBaseAddress;                                                                  /* PRQA S 2814 # JV-01 */

  /* Initialize the device-specific ADC hardware registers to their default values */
  LpAdcRegisters->usADCXnDFASENTSGER = ADC_WORD_ZERO;                                                                   /* PRQA S 2814 # JV-01 */
  LpAdcRegisters->ucADCXnSDVCLMINTER = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR0 = ADC_DWORD_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR1 = ADC_DWORD_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR0  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR1  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR2  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR0     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR1     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR2     = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGCR    = ADC_DWORD_ZERO;

  #if (ADC_ENABLE_ADC_TSEL == STD_ON)
  /* Check if HW unit has ADC trigger selection register */
  if (NULL_PTR != LpHwUnitConfig->pSgTriggReg)
  {
    /* De-initialization of ADC trigger selection register */
    Adc_HwTriggerDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif /* (ADC_ENABLE_ADC_TSEL == STD_ON) */

  #if (ADC_TRACK_AND_HOLD == STD_ON)
  /* Check if HW unit has Track and Hold groups */
  if ((uint8)0U != LpHwUnitConfig->ucTrackHoldEnable)
  {
    /* De-initialization of Track and Hold */
    Adc_TrackHoldDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif

  /* Check if HW unit has wait time configuration */
  if (ADC_WAITTIME_DISABLED != LpHwUnitConfig->ucWaitTimeIndex)
  {
    /* De-initialization of wait time configuration */
    Adc_DeInitWaitTime(LucHwUnitIndex);
  } /* else: No action required */
}
### Output Test Case:
"""

The `eval_prompt` I used was:

In [19]:
print(generate_response(eval_prompt, model, eval_tokenizer))
%time

<｜begin▁of▁sentence｜>No prerequisite actions are needed before generating this output test case as we are not operating on any global variables or system's shared objects here. We need just verify our driver by observing its normal operation flow through assertions/verification traps at each line of code execution after it’s executed without errors. It should perform de-initialization properly from a valid input by setting all used Register addresses to default known states.
The expected observations would be that memory sections pointed to by hardward abstraction layer have been zeroed out except for specific optional adcs related structures like ADC_TSEL which depends totally based off config settings in ADC_Initialise() setup phase when enabling relevant features with STM configurations.
Also these changes take place within scope of single testcase thus making them ideal since they won’t break other parts of software during cleanup sequence due to high level functionalities tests. T

Observe how the model does out of the box.

### 4. Set Up LoRA

Now, to start our fine-tuning, we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [15]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Let's print the model to examine its layers, as we will apply QLoRA to all the linear layers of the model. Those layers are `q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`, and `lm_head`.

In [17]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (n

Here we define the LoRA config.

`r` is the rank of the low-rank matrix used in the adapters, which thus controls the number of parameters trained. A higher rank will allow for more expressivity, but there is a compute tradeoff.

`alpha` is the scaling factor for the learned weights. The weight matrix is scaled by `alpha/r`, and thus a higher value for `alpha` assigns more weight to the LoRA activations.

The values used in the QLoRA paper were `r=64` and `lora_alpha=16`, and these are said to generalize well, but we will use `r=32` and `lora_alpha=64` so that we have more emphasis on the new fine-tuned data while also reducing computational complexity.

In [18]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 81117184 || all params: 3583627264 || trainable%: 2.2635496948825535


See how the model looks different now, with the LoRA adapters added:

In [19]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(

### Accelerator

Set up the Accelerator. I'm not sure if we really need this for a QLoRA given its [description](https://huggingface.co/docs/accelerate/v0.19.0/en/usage_guides/fsdp) (I have to read more about it) but it seems it can't hurt, and it's helpful to have the code for future reference. You can always comment out the accelerator if you want to try without.

In [36]:
#from accelerate import FullyShardedDataParallelPlugin, Accelerator
#from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
#
#fsdp_plugin = FullyShardedDataParallelPlugin(
#    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
#    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
#)
#
#accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [37]:
#model = accelerator.prepare_model(model)


Let's use Weights & Biases to track our training metrics. You'll need to apply an API key when prompted. Feel free to skip this if you'd like, and just comment out the `wandb` parameters in the `Trainer` definition below.

### 5. Run Training!

I didn't have a lot of training samples: only about 200 total train/validation. I used 500 training steps, and I was fine with overfitting in this case. I found that the end product worked well. It took about 20 minutes on the 1x A10G 24GB.

Overfitting is when the validation loss goes up (bad) while the training loss goes down significantly, meaning the model is learning the training set really well, but is unable to generalize to new datapoints. In most cases, this is not desired, but since I am just playing around with a model to generate outputs like my journal entries, I was fine with a moderate amount of overfitting.

With that said, a note on training: you can set the `max_steps` to be high initially, and examine at what step your model's performance starts to degrade. There is where you'll find a sweet spot for how many steps to perform. For example, say you start with 1000 steps, and find that at around 500 steps the model starts overfitting, as described above. Therefore, 500 steps would be your sweet spot, so you would use the `checkpoint-500` model repo in your output dir (`mistral-journal-finetune`) as your final model in step 6 below.

If you're just doing something for fun like I did and are OK with overfitting, you can try different checkpoint versions with different degrees of overfitting.

You can interrupt the process via Kernel -> Interrupt Kernel in the top nav bar once you realize you didn't need to train anymore.

In [20]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [21]:
import transformers
from datetime import datetime

project = "LLMs-HyLa-finetune"
base_model_name = "deepseek"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name



training_args = transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=900,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        #report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    )

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [22]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
25,0.927400,0.814959
50,0.732500,0.640160
75,0.632800,0.554897
100,0.516100,0.510050
125,0.490600,0.479664
150,0.475800,0.453240
175,0.428700,0.432620
200,0.413600,0.414841
225,0.409800,0.400658
250,0.385100,0.387730


C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLMs\venv\Lib\site-packages\peft\utils\save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLMs\venv\Lib\site-packages\peft\utils\save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLMs\venv\Lib\site-packages\peft\utils\save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
C:\Users\rvc_sw_mss1_common\Desktop\Fine_Tune_LLM

TrainOutput(global_step=900, training_loss=0.3810991170671251, metrics={'train_runtime': 22334.2029, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.04, 'total_flos': 1.80616753152e+17, 'train_loss': 0.3810991170671251, 'epoch': 0.1})

### 

I cleared the output of the cell above because I stopped the training early, and it produced a long, ugly error message.

### 6. Drum Roll... Try the Trained Model!

It's a good idea to kill the current process so that you don't run out of memory loading the base model again on top of the model we just trained. Go to `Kernel > Restart Kernel` or kill the process via the Terminal (`nvidia smi` > `kill [PID]`).

By default, the PEFT library will only save the QLoRA adapters, so we need to first load the base model from the Huggingface Hub:

Now load the QLoRA adapter from the appropriate checkpoint directory, i.e. the best performing model checkpoint:

In [24]:
clear_torch_cache()

In [25]:
print(model_checkpoint)
base_model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    device_map='auto',
    use_cache=False,
    quantization_config=nf4_config
    )#, quantization_config=bnb_config)
#model = AutoModelForCausalLM.from_pretrained(model_checkpoint, quantization_config=bnb_config)

C:\Users\rvc_sw_mss1_common\Desktop\test\mcal-guru\model\deepseek-aideepseek-coder-6.7b-instruct


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.73s/it]


In [27]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "deepseek-LLMs-HyLa-finetune/checkpoint-900")

and run your inference!

Let's try the same `eval_prompt` and thus `model_input` as above, and see if the new finetuned model performs better. I like playing with the repetition penalty (just little tweaks of .01-.05 at a time). THIS IS SO FUN. I'm obsessed wth this AI version of myself.

In [28]:
ft_tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    add_eos_token=True,
    add_bos_token=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
def generate_response_ft(prompt, model, tokenizer):
    with torch.no_grad():
        encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
        model_inputs = encoded_input.to("cuda")
    
        generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, repetition_penalty=1.15, temperature=0.2)
        decoded_output = tokenizer.batch_decode(generated_ids,  skip_special_tokens=True)
    clear_torch_cache()

    return decoded_output[0].replace(prompt, "")

In [30]:
import time

### Checkpoint-500

In [32]:
%%time
eval_prompt = """
### Read the below Input Function, then generate the appropriate Output Test Case.\n ### Input Function: 
** Function Name        : Adc_AdcDeInit
**
** Service ID           : NA
**
** Description          : This internal function performs the de-initialization of the device-specific ADC hardware
**                        registers.
**
** Sync/Async           : Synchronous
**
** Re-entrancy          : Non-Reentrant
**
** Input Parameters     : LucHwUnitIndex
**
** InOut Parameters     : None
**
** Output Parameters    : None
**
** Return Parameter     : None
**
** Preconditions        : None
**
** Global Variables     : Adc_GpHwUnitConfig
**
** Functions invoked    : Adc_HwTriggerDeInit, Adc_TrackHoldDeInit, Adc_DeInitWaitTime
**
** Registers Used       : ADCXnDFASENTSGER, ADCXnSDVCLMINTER, ADCXnSGDIAGPCCR0, ADCXnSGDIAGPCCR1, ADCXnSGDIAGVCR0,
**                        ADCXnSGDIAGVCR1, ADCXnSGDIAGVCR2, ADCXnSTPDCR0, ADCXnSTPDCR1, ADCXnSTPDCR2, ADCXnSGDIAGCR
**
** Reference ID         : ADC_DUD_ACT_085, ADC_DUD_ACT_085_REG001, ADC_DUD_ACT_085_REG002,
** Reference ID         : ADC_DUD_ACT_085_REG003, ADC_DUD_ACT_085_REG004, ADC_DUD_ACT_085_REG005,
** Reference ID         : ADC_DUD_ACT_085_REG006, ADC_DUD_ACT_085_REG007, ADC_DUD_ACT_085_REG008,
** Reference ID         : ADC_DUD_ACT_085_REG009, ADC_DUD_ACT_085_REG010, ADC_DUD_ACT_085_REG011
***********************************************************************************************************************/
#if (ADC_DEINIT_API == STD_ON)

#define ADC_START_SEC_PRIVATE_CODE
#include "Adc_Mapping.h"                                                                                                /* PRQA S 5087 # JV-01 */

FUNC(void, ADC_PRIVATE_CODE) Adc_AdcDeInit(const uint8 LucHwUnitIndex)                                                  /* PRQA S 1532 # JV-01 */
{
  P2CONST(Adc_HwUnitConfigType, AUTOMATIC, ADC_CONFIG_DATA) LpHwUnitConfig;
  P2VAR(volatile Adc_ConfigRegisters, AUTOMATIC, REGSPACE) LpAdcRegisters;

  /* Initialize the local variable for HW unit configuration */
  LpHwUnitConfig = &Adc_GpHwUnitConfig[LucHwUnitIndex];                                                                 /* PRQA S 2824 # JV-01 */
  /* Read the user base configuration address of the HW unit */
  LpAdcRegisters = LpHwUnitConfig->pHwUnitBaseAddress;                                                                  /* PRQA S 2814 # JV-01 */

  /* Initialize the device-specific ADC hardware registers to their default values */
  LpAdcRegisters->usADCXnDFASENTSGER = ADC_WORD_ZERO;                                                                   /* PRQA S 2814 # JV-01 */
  LpAdcRegisters->ucADCXnSDVCLMINTER = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR0 = ADC_DWORD_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR1 = ADC_DWORD_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR0  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR1  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR2  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR0     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR1     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR2     = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGCR    = ADC_DWORD_ZERO;

  #if (ADC_ENABLE_ADC_TSEL == STD_ON)
  /* Check if HW unit has ADC trigger selection register */
  if (NULL_PTR != LpHwUnitConfig->pSgTriggReg)
  {
    /* De-initialization of ADC trigger selection register */
    Adc_HwTriggerDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif /* (ADC_ENABLE_ADC_TSEL == STD_ON) */

  #if (ADC_TRACK_AND_HOLD == STD_ON)
  /* Check if HW unit has Track and Hold groups */
  if ((uint8)0U != LpHwUnitConfig->ucTrackHoldEnable)
  {
    /* De-initialization of Track and Hold */
    Adc_TrackHoldDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif

  /* Check if HW unit has wait time configuration */
  if (ADC_WAITTIME_DISABLED != LpHwUnitConfig->ucWaitTimeIndex)
  {
    /* De-initialization of wait time configuration */
    Adc_DeInitWaitTime(LucHwUnitIndex);
  } /* else: No action required */
}
### Output Test Case:
"""

generate_response_ft(eval_prompt, ft_model, ft_tokenizer)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


CPU times: total: 6min 30s
Wall time: 3min 26s


'/* ****************************************************************************\n ** File Name: Adc_GetGroupStatus.c\n **\n ** Date Created: 2018-09-20\n **\n ** Version: 1.0\n **\n ** Device Class: V4H\n **\n ** Description: Contains CAT2 interrupt processing functions for ADC Driver Component\n **              These are reentrant functions which can be used by other driver components\n **              or external interrupts.\n **\n ** --------------------------------------------------------------------------\n ** Modification History:\n ** Release Number (DD/MM/YYYY) - Description of Change\n ** ------- --- ------------------ --------------------------------------------\n ** 1.0      DD/MM/YYYY Initial version of file added\n ** 2.0      DD/MM/YYYY Updated according to the new header structure\n ** 3.0      DD/MM/YYYY Removed unused input parameter \'blStartConversion\' from API \\\n **                      Adc_CheckError and Adc_SetMode()\n ** 4.0      DD/MM/YYYY Added support for 

In [35]:
eval_prompt = """
### Read the below Input Function, then generate the appropriate Output Test Case.\n ### Input Function: 
Function Name        : Adc_AdcDeInit
Service ID           : NA
Description          : This internal function performs the de-initialization of the device-specific ADC hardware registers.
Sync/Async           : Synchronous
Re-entrancy          : Non-Reentrant
Input Parameters     : LucHwUnitIndex
InOut Parameters     : None
Output Parameters    : None
Return Parameter     : None
Preconditions        : None
Global Variables     : Adc_GpHwUnitConfig
Functions invoked    : Adc_HwTriggerDeInit, Adc_TrackHoldDeInit, Adc_DeInitWaitTime
Registers Used       : ADCXnDFASENTSGER, ADCXnSDVCLMINTER, ADCXnSGDIAGPCCR0, ADCXnSGDIAGPCCR1, ADCXnSGDIAGVCR0, ADCXnSGDIAGVCR1, ADCXnSGDIAGVCR2, ADCXnSTPDCR0, ADCXnSTPDCR1, ADCXnSTPDCR2, ADCXnSGDIAGCR
Reference ID         : ADC_DUD_ACT_085, ADC_DUD_ACT_085_REG001, ADC_DUD_ACT_085_REG002,
Reference ID         : ADC_DUD_ACT_085_REG003, ADC_DUD_ACT_085_REG004, ADC_DUD_ACT_085_REG005,
Reference ID         : ADC_DUD_ACT_085_REG006, ADC_DUD_ACT_085_REG007, ADC_DUD_ACT_085_REG008,
Reference ID         : ADC_DUD_ACT_085_REG009, ADC_DUD_ACT_085_REG010, ADC_DUD_ACT_085_REG011
#if (ADC_DEINIT_API == STD_ON)

#define ADC_START_SEC_PRIVATE_CODE
#include "Adc_Mapping.h"                                                                                                /* PRQA S 5087 # JV-01 */

FUNC(void, ADC_PRIVATE_CODE) Adc_AdcDeInit(const uint8 LucHwUnitIndex)                                                  /* PRQA S 1532 # JV-01 */
{
  P2CONST(Adc_HwUnitConfigType, AUTOMATIC, ADC_CONFIG_DATA) LpHwUnitConfig;
  P2VAR(volatile Adc_ConfigRegisters, AUTOMATIC, REGSPACE) LpAdcRegisters;

  /* Initialize the local variable for HW unit configuration */
  LpHwUnitConfig = &Adc_GpHwUnitConfig[LucHwUnitIndex];                                                                 /* PRQA S 2824 # JV-01 */
  /* Read the user base configuration address of the HW unit */
  LpAdcRegisters = LpHwUnitConfig->pHwUnitBaseAddress;                                                                  /* PRQA S 2814 # JV-01 */

  /* Initialize the device-specific ADC hardware registers to their default values */
  LpAdcRegisters->usADCXnDFASENTSGER = ADC_WORD_ZERO;                                                                   /* PRQA S 2814 # JV-01 */
  LpAdcRegisters->ucADCXnSDVCLMINTER = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR0 = ADC_DWORD_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR1 = ADC_DWORD_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR0  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR1  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR2  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR0     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR1     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR2     = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGCR    = ADC_DWORD_ZERO;

  #if (ADC_ENABLE_ADC_TSEL == STD_ON)
  /* Check if HW unit has ADC trigger selection register */
  if (NULL_PTR != LpHwUnitConfig->pSgTriggReg)
  {
    /* De-initialization of ADC trigger selection register */
    Adc_HwTriggerDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif /* (ADC_ENABLE_ADC_TSEL == STD_ON) */

  #if (ADC_TRACK_AND_HOLD == STD_ON)
  /* Check if HW unit has Track and Hold groups */
  if ((uint8)0U != LpHwUnitConfig->ucTrackHoldEnable)
  {
    /* De-initialization of Track and Hold */
    Adc_TrackHoldDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif

  /* Check if HW unit has wait time configuration */
  if (ADC_WAITTIME_DISABLED != LpHwUnitConfig->ucWaitTimeIndex)
  {
    /* De-initialization of wait time configuration */
    Adc_DeInitWaitTime(LucHwUnitIndex);
  } /* else: No action required */
}
### Output Test Case:
"""

generate_response_ft(eval_prompt, ft_model, ft_tokenizer)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'/* ****************************************************************************\n** File Name: adc_dataindicate_test.c\n **\n ** Date: 2017-08-08\n **\n ** Description: Contains test cases for functions used in Data Indication feature.\n **\n ** Functions Included: Adc_DataIndicate()\n ** /\n #include "adc_test_common.h"\n #include "adc_dataindicate_test.h"\n\n static void Adc_TestSetUp(void);\n static void Adc_CommonTestSetup(void);\n static void Adc_CheckErrorStatusAndClearFlagIfAnyErrrorOccurred(boolean blExpectedResult);\n static boolean Adc_GetGroupIdFromChannelID(uint8 ChannelIdx);\n static boolean Adc_CompareGroupsWithChannels(uint8 GroupIdx, const Adc_GroupInfoType* LpGrpPtr);\n static Std_ReturnType Adc_CallApiToReadRegisterValue(Std_DomainType Domain, uint32 RegisterOffset, uint32 Value);\n static Std_ReturnType Adc_CallApiToWriteRegisterValue(Std_DomainType Domain, uint32 RegisterOffset, uint32 Mask, uint32 Value);\n static Std_ReturnType Adc_CallApiToStartConversionMode(St

In [36]:
eval_prompt = """
### Read the below Input Function, then generate the appropriate Output Test Case.\n ### Input Function: 
#if (ADC_DEINIT_API == STD_ON)

#define ADC_START_SEC_PRIVATE_CODE
#include "Adc_Mapping.h"                                                                                                /* PRQA S 5087 # JV-01 */

FUNC(void, ADC_PRIVATE_CODE) Adc_AdcDeInit(const uint8 LucHwUnitIndex)                                                  /* PRQA S 1532 # JV-01 */
{
  P2CONST(Adc_HwUnitConfigType, AUTOMATIC, ADC_CONFIG_DATA) LpHwUnitConfig;
  P2VAR(volatile Adc_ConfigRegisters, AUTOMATIC, REGSPACE) LpAdcRegisters;

  /* Initialize the local variable for HW unit configuration */
  LpHwUnitConfig = &Adc_GpHwUnitConfig[LucHwUnitIndex];                                                                 /* PRQA S 2824 # JV-01 */
  /* Read the user base configuration address of the HW unit */
  LpAdcRegisters = LpHwUnitConfig->pHwUnitBaseAddress;                                                                  /* PRQA S 2814 # JV-01 */

  /* Initialize the device-specific ADC hardware registers to their default values */
  LpAdcRegisters->usADCXnDFASENTSGER = ADC_WORD_ZERO;                                                                   /* PRQA S 2814 # JV-01 */
  LpAdcRegisters->ucADCXnSDVCLMINTER = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR0 = ADC_DWORD_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGPCCR1 = ADC_DWORD_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR0  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR1  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSGDIAGVCR2  = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR0     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR1     = ADC_BYTE_ZERO;
  LpAdcRegisters->ucADCXnSTPDCR2     = ADC_BYTE_ZERO;
  LpAdcRegisters->ulADCXnSGDIAGCR    = ADC_DWORD_ZERO;

  #if (ADC_ENABLE_ADC_TSEL == STD_ON)
  /* Check if HW unit has ADC trigger selection register */
  if (NULL_PTR != LpHwUnitConfig->pSgTriggReg)
  {
    /* De-initialization of ADC trigger selection register */
    Adc_HwTriggerDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif /* (ADC_ENABLE_ADC_TSEL == STD_ON) */

  #if (ADC_TRACK_AND_HOLD == STD_ON)
  /* Check if HW unit has Track and Hold groups */
  if ((uint8)0U != LpHwUnitConfig->ucTrackHoldEnable)
  {
    /* De-initialization of Track and Hold */
    Adc_TrackHoldDeInit(LucHwUnitIndex);
  } /* else: No action required */
  #endif

  /* Check if HW unit has wait time configuration */
  if (ADC_WAITTIME_DISABLED != LpHwUnitConfig->ucWaitTimeIndex)
  {
    /* De-initialization of wait time configuration */
    Adc_DeInitWaitTime(LucHwUnitIndex);
  } /* else: No action required */
}
### Output Test Case:
"""

generate_response_ft(eval_prompt, ft_model, ft_tokenizer)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"/* This is a test case which tests whether all internal variables are initialized properly or not during invoking API function 'Adc_ChannelGroupDisableNotification()'.*/\n\n/*** Test Step ***/\n1. Call the testing API with valid parameters.\n2. Invoke the target API under test without any parameter check.\n\n/*** Expected Result ***/\nThe return value should be E_OK since no error occurred in this case. The notification status of each channel group must have been set as ADC_FALSE.\n\n/*** Variables Used In Testing ***/\n&map_Adc_GaaNotifyStatus[0]<\\n>= ADC_TRUE;\\n&map_Adc_GaaNotifyStatus[1]<\\n>= ADC_TRUE;\\n&map_Adc_GaaNotifyStatus[2]<\\n>= ADC_TRUE;\\n&map_Adc_GaaNotifyStatus[3]<\\n>= ADC_TRUE;\n\n/*** Functions Used In Testing ***/\nNone\n\n/*** Registers Used In Testing ***/\nNone\n\n/*** Code Used In Testing ***/\nApiCalled : VARIABLE_TESTED = ADC_TRUE; \\nreturn (E_NOT_OK); // Return value verification will fail because it's expected to return E_OK instead of E_NOT_OK.\n\n// G